In [2]:
import csv
import pandas as pd
import os

The following block is an example block to show how to read a csv and preview first several lines.

In [36]:
'''
Get star of a shop in each month
:param business_id: The shop's id
:returns: Star of the shop every month as a list, the start month and the end month
'''
business_id = 'e4Vwtrqf-wpJfwesgvdgxQ'
df = pd.read_csv("./new_orlean_restaurant_reviews.csv")
target_bus_df = df.loc[df['business_id'] == business_id]
print("number of reviews: ",len(target_bus_df.index))
target_bus_df = target_bus_df.sort_values(by=['date'])

start_year = int(target_bus_df.iloc[0].at['date'][0:4])
end_year = int(target_bus_df.iloc[-1].at['date'][0:4])
start_month = int(target_bus_df.iloc[0].at['date'][5:7])
end_month = int(target_bus_df.iloc[-1].at['date'][5:7])
# star - month dictionary re-scaled to 0 - (month_count-1)
star_month_dict = {}
for i in range(0,len(target_bus_df.index)):
    real_year = int(target_bus_df.iloc[i].at['date'][0:4])
    real_month = int(target_bus_df.iloc[i].at['date'][5:7])
    key_month = (real_year - start_year) * 12 + (real_month - start_month)
    if key_month in star_month_dict:
        star_month_dict[key_month].append(target_bus_df.iloc[i].at['stars'])
    else:
        star_month_dict[key_month] = [target_bus_df.iloc[i].at['stars']]
print(star_month_dict)
# ensure every month has a average star
month_count = (end_year - start_year) * 12 + (end_month - start_month) + 1
per_month_stars = []
for key_month in range(month_count):
    if key_month in star_month_dict:
        per_month_stars.append(sum(star_month_dict[key_month])/len(star_month_dict[key_month]))
    else:
        per_month_stars.append(per_month_stars[key_month-1])

print (per_month_stars, start_year, start_month, end_year, end_month)
target_bus_df


number of reviews:  35
{0: [5.0, 5.0, 5.0], 1: [4.0, 4.0, 3.0, 5.0, 5.0, 5.0], 3: [5.0, 5.0, 4.0], 4: [5.0, 4.0, 5.0], 5: [5.0], 6: [4.0], 7: [1.0], 8: [5.0, 5.0], 9: [5.0], 10: [3.0], 11: [1.0], 12: [3.0, 5.0], 13: [2.0, 4.0], 14: [4.0, 5.0], 15: [5.0, 5.0, 3.0, 5.0], 17: [1.0], 19: [4.0]}
[5.0, 4.333333333333333, 4.333333333333333, 4.666666666666667, 4.666666666666667, 5.0, 4.0, 1.0, 5.0, 5.0, 3.0, 1.0, 4.0, 3.0, 4.5, 4.5, 4.5, 1.0, 1.0, 4.0] 2016 12 2018 7


,funny,useful,review_id,text,business_id,stars,date,user_id,cool
18263,0,2,SpnXhe2dfsVqf1-zToMqDQ,"Just devoured the mama's melt!! Soooo good, th...",e4Vwtrqf-wpJfwesgvdgxQ,5.0,2016-12-13 23:25:20,GpmVD0JqYBELqF5F0e-JCQ,1
32020,0,1,qsqsMSb9SGSU1rES1Ho7AA,"Saw Melt as a ""hot and new"" restaurant on Yelp...",e4Vwtrqf-wpJfwesgvdgxQ,5.0,2016-12-31 23:00:11,QcTi1zVIsOdmb5_KFqilew,1
40956,0,3,nGF4R21QRP3jkZYttyB6nA,Stopped in at Melt on 12/30 for a late lunch (...,e4Vwtrqf-wpJfwesgvdgxQ,5.0,2016-12-31 23:10:34,afapBXYTOu34mw_ZFbJd3w,1
44850,0,2,-ztekvXtRc5I69oxk5e2dQ,This place is great! \n\nA wonderful light an...,e4Vwtrqf-wpJfwesgvdgxQ,4.0,2017-01-02 22:31:48,fBAoEWbN2mJYPEceDaCC6A,3
0,0,1,Sx8TMOWLNuJBWer-0pcmoA,Cute interior and owner (?) gave us tour of up...,e4Vwtrqf-wpJfwesgvdgxQ,4.0,2017-01-14 20:54:15,bcjbaE6dDog4jkNY91ncLQ,1
18409,0,1,UegH7kpH2zCR565KwFOM7g,Started with the cheese curds- not very curd-l...,e4Vwtrqf-wpJfwesgvdgxQ,3.0,2017-01-18 17:53:16,u_MVJXe6gg20QPecy0fAXQ,0
28276,0,1,GG2oSRacpEsvGpWVIUkhoQ,Wow what a meatball sub! Delicious!! Salad was...,e4Vwtrqf-wpJfwesgvdgxQ,5.0,2017-01-24 18:08:51,XGzHAWjkB5pIcPnjaLqggQ,0
8646,0,1,QgpdsGrJOAGQO2Ag2yW5QQ,Food is tasty. Service and management are flaw...,e4Vwtrqf-wpJfwesgvdgxQ,5.0,2017-01-30 18:23:14,xrjPo07UsqNj5eVT7Z7ZKw,0
7772,0,1,c-caXBb1n3yqMS2gqr6B3g,Fresh food and good atmosphere. Even the most ...,e4Vwtrqf-wpJfwesgvdgxQ,5.0,2017-01-31 18:06:07,Vc83QXHTM-L27BR11ca_LQ,0
20112,0,0,KV2cYQGMFevMj6bVClH2EQ,Everything was very very good. The chicken Par...,e4Vwtrqf-wpJfwesgvdgxQ,5.0,2017-03-02 20:49:26,eQRkfCXBJ-Z55xlEX8VE8g,0


In [7]:
gallery = PaperGallery()
gallery.construct_gallery_from_main_dataset()
gallery.gallery

NameError: name 'PaperGallery' is not defined

In [6]:
len(main_dataset)

3394